In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://nhacnheo.com/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
songs = []


In [46]:
import requests
from bs4 import BeautifulSoup

# Function to fetch HTML content
def fetch_html(url):
    response = requests.get(url)
    return response.content

# Function to extract data from the main table
def extract_data_from_table(url):
    html_content = fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    rows = soup.find('tbody').find_all('tr')
    results = []

    for row in rows:
        # Get the 2nd <td> link
        second_td = row.find_all('td')[1]
        second_link = second_td.find('a')['href']

        # Get the 4th <td> link
        fourth_td = row.find_all('td')[3]
        fourth_link = fourth_td.find('a')['href']

        # Fetch and process the images from the second link
        images = fetch_images_from_link(second_link)

        # Fetch and process details from the fourth link
        title, author, tags, lyrics = fetch_details_from_link(fourth_link)

        results.append({
            'second_link_images': images,
            'fourth_link_title': title,
            'fourth_link_author': author,
            'fourth_link_tags': tags,
            'fourth_link_lyrics': lyrics
        })

    return results

# Function to fetch images from a link
def fetch_images_from_link(url):
    html_content = fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    image_tags = soup.select('div.ct-box img')
    images = [img['src'] for img in image_tags]
    return images

# Function to fetch title, author, tags, and lyrics from a link
def fetch_details_from_link(url):
    html_content = fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract title
    title = soup.find('h1', class_='h3 d-inline-block').text.strip()

    # Extract author
    author_section = soup.find(text=lambda x: "Sáng tác" in x)
    author = ""
    if author_section:
        author = author_section.split("Sáng tác:")[1].split('|')[0].strip()

    # Extract tags
    tags_section = soup.find('i', class_='fas fa-book')
    tags = ""
    if tags_section:
        tags = tags_section.find_next('a').text.strip()

    # Extract lyrics
    lyrics_div = soup.find('div', id='lyric')
    if lyrics_div:
        # for span in lyrics_div.find_all('span'):
        #     span.extract()
        # Delete all the <span></span> content inside lyrics_div
        [span.extract() for span in lyrics_div.find_all('span')]
        
        lyrics = lyrics_div.text.strip()

    return title, author, tags, lyrics

# Main URL
url = "https://hopamviet.vn/sheet/index/"

# Get the results
data = extract_data_from_table(url)

# Output the results (you can save this to a file or process it further)
for item in data:
    print("Title:", item['fourth_link_title'])
    print("Author:", item['fourth_link_author'])
    print("Tags:", item['fourth_link_tags'])
    print("Lyrics:", item['fourth_link_lyrics'])
    print("Images:", item['second_link_images'])
    print("-" * 40)


C:\Users\hanhn\AppData\Local\Temp\ipykernel_3800\3016634967.py:60: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  author_section = soup.find(text=lambda x: "Sáng tác" in x)


NameError: name 'images' is not defined

In [70]:
import requests
from bs4 import BeautifulSoup
import re

# Function to fetch HTML content
def fetch_html(url):
    response = requests.get(url)
    return response.content

# Function to extract data from a specific <tr>
def extract_data_from_row(url, row_index=0):
    html_content = fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find all <tr> elements in the <tbody>
    rows = soup.find('tbody').find_all('tr')
    
    if not rows or row_index >= len(rows):
        print(f"No <tr> found at index {row_index}.")
        return
    
    # Get the specified <tr>
    row = rows[row_index]
    
    # Get the 2nd <td> link
    second_td = row.find_all('td')[1]
    second_link = second_td.find('a')['href']
    print("Second Link:", second_link)

    # Get the 4th <td> link
    fourth_td = row.find_all('td')[3]
    fourth_link = fourth_td.find('a')['href']
    print("Fourth Link:", fourth_link)

    # Fetch and process the images from the second link
    # images = fetch_images_from_link(second_link)
    files = fetch_files_from_link(second_link)
    # print("Images:", images)
    print("Files:", files)

    # Fetch and process details from the fourth link
    title, author, tags, lyrics = fetch_details_from_link(fourth_link)

    print("Title:", title)
    print("Author:", author)
    print("Tags:", tags)
    print("Lyrics:\n", lyrics)

# Function to fetch images and other files from a link
def fetch_files_from_link(url):
    html_content = fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Fetch image sources from <img> tags
    image_tags = soup.select('div.ct-box img')
    images = [img['src'] for img in image_tags]
    
    # Fetch PDF or other file sources from <iframe> tags
    iframe_tags = soup.select('div.ct-box iframe')
    files = []
    for iframe in iframe_tags:
        src = iframe.get('src')
        if src and "google.com/viewer?url=" in src:
            # Extract the actual file URL from the Google Viewer link
            file_url = src.split("url=")[1].split("&")[0]
            files.append(file_url)
    
    # Combine images and files into a single list
    files.extend(images)
    
    return files
# Function to fetch title, author, tags, and lyrics from a link
def fetch_details_from_link(url):
    html_content = fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract title
    title = soup.find('h1', class_='h3 d-inline-block').text.strip()

    # Extract author names
    author_section = soup.find('i', title="Sáng tác")
    author = ""
    if author_section:
        # Get the entire author text including the links
        author_text_node = author_section.find_next_sibling(text=True)
        if author_text_node:
            # Find all <a> tags before encountering the next "|" symbol
            author_links = []
            current = author_text_node.find_next_sibling('a')
            while current and current.name == 'a':
                author_links.append(current.text.strip())
                current = current.find_next_sibling()
                if isinstance(current, str) and '|' in current:
                    break
            author = ' & '.join(author_links)

    # Extract tags
    tags_section = soup.find('i', class_='fas fa-book')
    tags = ""
    if tags_section:
        tags = tags_section.find_next('a').text.strip()

    # Extract lyrics
    lyrics_div = soup.find('div', id='lyric')
    lyrics = lyrics_div.text.strip()
    
    # Remove all chords in the format [Am], [Dm], etc. using regex
    lyrics = re.sub(r'\[.*?\] ', '', lyrics)
    # FInd if in the lyrics there are any chords in the format [Am], [Dm], etc.
    # chords = re.findall(r'\[.*?\]', lyrics)
    # Remove all of them
    lyrics = re.sub(r'\[.*?\]', '', lyrics)    

    return title, author, tags, lyrics

# Main URL
# url = "https://hopamviet.vn/sheet/index/"
url = "https://hopamviet.vn/sheet/index/700"

# Test with a specific <tr> for debugging
# For example, test with the second <tr> (index 1)
extract_data_from_row(url, row_index=99)


Second Link: https://hopamviet.vn/sheet/song/chi-anh-moi-hieu/W8IU9WWD.html
Fourth Link: https://hopamviet.vn/chord/song/chi-anh-moi-hieu/W8IU9WWD.html
Files: ['https://hopamviet.vn/assets/images/sheets/CHI-MINH-ANH-HIEUpdf.pdf']
Title: Chỉ anh mới hiểu
Author: Nguyễn Đình Chương
Tags: Nhạc Trữ tình
Lyrics:
 Cuộc tình theo gió cuốn về đâu cho thêm sầu
Còn lại nỗi nhớ biết tìm em ở chốn nao
Lời thề nguyện ước sẽ không xa rời nhau
Anh vẫn nhớ em người yêu dấu thế nhưng em xa rồi

Người ở nơi ấy có buồn không cho cuộc tình
Cuộc tình tan vỡ hỏi tại sao lòng đắng cay
Chờ hoài chẳng thấy bóng dáng em quay về đây
Khi bóng đêm lạnh lùng.

ĐK
Người yêu ơi người biết không hạnh phúc anh chỉ cần có em
Chỉ mình em và em thôi tình yêu ấy chỉ anh mới hiểu được tình anh
Đến thế nào mà em ơi em còn nhớ về anh

Người yêu ơi dù cách xa nhưng trái tim vẫn còn nhớ em
Nhớ về em dẫu thế nào dù hai đứa hai lối về 
Tình vỡ chẳng còn thì anh vẫn 
Trọn đời mãi yêu em

CODA
Em dù ở đâu tình chúng ta cách chia đô

C:\Users\hanhn\AppData\Local\Temp\ipykernel_3800\1039865819.py:85: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  author_text_node = author_section.find_next_sibling(text=True)
